<a href="https://colab.research.google.com/github/Ddkaba/IAD_Lab_2/blob/main/IAD_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Группа 4232

Спицов А.

Михайлов Д.

Вариант №2 (Анализ вин по трем производителям)

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("https://raw.githubusercontent.com/Ddkaba/IAD_Lab_2/main/V2_classification_lr3.csv", index_col=0)
if 'No' in dataset.columns:
    dataset = dataset.drop(columns=['No'])

In [ ]:
TARGET = 'target'

print("Общая информация")
print(dataset.info())

print(f"Количество записей (объектов): {dataset.shape[0]}")
print(f"Количество признаков (фич): {dataset.shape[1]}")

print("\nНазвания столбцов:")
print(dataset.columns.tolist())

print("\nТипы данных:")
print(dataset.dtypes)

print("\nПропущенные значения:")
missing_values = dataset.isnull().sum()
print(missing_values)
print(f"Общее количество пропущенных значений: {missing_values.sum()}")

print("Целевая переменная")
if TARGET in dataset.columns:
    print(f"\nЦелевая переменная: {TARGET}")
    print(f"Тип данных целевой переменной: {dataset[TARGET].dtype}")
    unique_values = dataset[TARGET].unique()
    print(f"Уникальные значения целевой переменной (первые 20): {unique_values[:20]}")
    print(f"Всего уникальных значений: {unique_values.size}")
    if dataset[TARGET].nunique() <= 20:
        print("Распределение классов:")
        print(dataset[TARGET].value_counts())
        print("Процентное соотношение классов:")
        print(dataset[TARGET].value_counts(normalize=True) * 100)

print("Статистика")
print(dataset.describe())

print("Анализ кат. признаков")
categorical_features = []
for col in dataset.columns:
    unique_values = dataset[col].nunique(dropna=True)
    if unique_values <= 10:
        categorical_features.append(col)
        print(f"{col}: {unique_values} уникальных значений - {dataset[col].unique()}")

print(f"\nВсего категориальных признаков: {len(categorical_features)}")

Общая информация
<class 'pandas.core.frame.DataFrame'>
Index: 178 entries, 14.23 to 14.13
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   malic_acid                    178 non-null    float64
 1   ash                           178 non-null    float64
 2   alcalinity_of_ash             178 non-null    float64
 3   magnesium                     178 non-null    float64
 4   total_phenols                 178 non-null    float64
 5   flavanoids                    178 non-null    float64
 6   nonflavanoid_phenols          178 non-null    float64
 7   proanthocyanins               178 non-null    float64
 8   color_intensity               178 non-null    float64
 9   hue                           178 non-null    float64
 10  od280/od315_of_diluted_wines  178 non-null    float64
 11  proline                       178 non-null    float64
 12  target                        178 non-null    

In [ ]:
# Корреляционная матрица для Wine датасета
correlation_matrix = dataset.corr(numeric_only=True)
plt.figure(figsize=(12, 10))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Корреляционная матрица признаков Wine')
plt.tight_layout()
plt.show()

In [ ]:
# Гистограммы по всем числовым признакам
# Определим список числовых колонок и исключим целевую, если она есть
feature_columns = dataset.select_dtypes(include=[np.number]).columns.tolist()
if 'target' in feature_columns:
    feature_columns.remove('target')

_ = dataset[feature_columns].hist(
    bins=10,
    figsize=(20, 15),
    grid=False,
    edgecolor='black'
)
plt.suptitle('Распределение числовых признаков', y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
df_original = dataset.copy()

In [ ]:
# Использование SelectKBest для оценки информативности признаков и выбора топ-5 (Wine)
source_df = dataset

# Числовые признаки
X_num = source_df.drop(columns=[TARGET]).select_dtypes(include=[np.number])
y = source_df[TARGET]

# Импутация оставшихся пропусков медианой (на всякий случай)
imputer = SimpleImputer(strategy='median')
X_num_imp = pd.DataFrame(imputer.fit_transform(X_num), columns=X_num.columns, index=X_num.index)

# Для классификации используем f_classif
all_selector = SelectKBest(score_func=f_classif, k='all')
all_selector.fit(X_num_imp, y)

# Результаты
scores_df = (
    pd.DataFrame({'feature': X_num_imp.columns, 'score': all_selector.scores_})
      .sort_values('score', ascending=False)
      .reset_index(drop=True)
)

print('Оценки информативности (f_classif), по убыванию:')
print(scores_df)

plt.figure(figsize=(10, 6))
sns.barplot(data=scores_df, x='score', y='feature', color='#1f77b4')
plt.title('SelectKBest: f_classif scores (Wine)')
plt.tight_layout()
plt.show()

# Выбор ТОП-K признаков
K = 5
selector = SelectKBest(score_func=f_classif, k=K)
selector.fit(X_num_imp, y)
selected_features = X_num_imp.columns[selector.get_support()].tolist()
print(f'Топ-{K} признаков:')
print(selected_features)


In [ ]:
# Инженерия признаков для Wine: доля флавоноидов в общих фенолах
EPS = 1e-9
dataset['flavonoid_share'] = dataset['flavanoids'] / (dataset['total_phenols'] + EPS)

# Проверка наличия целевой переменной в корреляционной матрице и вывод топ-корреляций
corr = dataset.corr()
if TARGET in corr.columns:
    print('\nТоп корреляций с целевой переменной:')
    print(corr[TARGET].sort_values(ascending=False).head(15))

In [ ]:
# Удаление нерелевантных признаков: ash и magnesium, затем корреляционная матрица
cols_to_drop = ['ash', 'magnesium']
actual_drop = [c for c in cols_to_drop if c in dataset.columns]
print('Удаляем признаки:', actual_drop)

dataset = dataset.drop(columns=actual_drop)
print('После удаления:', dataset.shape)
print('Текущие столбцы:')
print(dataset.columns.tolist())

# Корреляционная матрица после удаления
correlation_matrix = dataset.corr(numeric_only=True)
plt.figure(figsize=(12, 10))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Корреляционная матрица после удаления (Wine)')
plt.tight_layout()
plt.show()